# Informe Hito 2 Mineria de Datos - Grupo 16
## Integrantes:
- Constanza Dominguez ()
- Felipe Caballero (Resultados (P3), Introducción)
- Jose Pablo Canales ()
- Sebastian Salas (Resultados (P3), Metodología Experimental)
- Vicente Arce ()

## 1. Introduccion

En las últimas decadas, la industria de los videojuegos ha experimentado una expansión sin precedentes, tanto en términos de volumen de producción como en la diversidad de públicos que la consumen. Hoy en día, los videojuegos constituyen una de las formas de entretenimiento más influyentes a nivel global, con miles de títulos disponibles en múltiples plataformas y una comunidad cada vez más diversa, que abarca desde niños hasta adultos mayores. Este fenómeno cultural ha evolucionado más allá del ocio, incorporando espacios de socialización, desarrollo cognitivo, educación e incluso bienestar emocional.

Dentro de este ecosistema destaca la práctica del speedrunning, que consiste en completar videojuegos en el menor tiempo posible bajo reglas establecidas por la propia comunidad. Lo que comenzó como una forma alternativa de jugar, se ha transformado en un fenómeno cultural masivo, con competencias internacionales, transmisiones en vivo y una comunidad activa que documenta estrategias, tiempos y experiencias compartidas.

En este contexto, el sitio web HowLongToBeat ofrece un punto de observación privilegiado, al reunir estadísticas de comportamiento de usuarios en relación con miles de videojuegos, incluyendo estimaciones de duración, porcentajes de finalización, plataformas jugadas, valoraciones, y tiempos promedio de juego bajo distintos estilos. El análisis de este tipo de datos permite estudiar no solo las propiedades inherentes de los juegos, sino también los patrones de uso y consumo en una comunidad global de jugadores.

Inicialmente, nuestro interés se centró en analizar un dataset relacionado con speedruns, dada su buena calidad y potencial para observar rendimientos comparativos. Sin embargo, al profundizar en el análisis, decidimos ampliar el enfoque hacia una mirada más integral del comportamiento del usuario en la plataforma, considerando no solo el rendimiento, sino también la relación entre atributos del juego y conductas agregadas de los jugadores.

En este estudio nos propusimos dos objetivos principales:

- Predecir ciertos comportamientos esperados de usuarios (como el tiempo de juego o la valoración) a partir de variables definidas dentro del conjunto de datos.

- Categorizar juegos en función de una combinación de atributos propios del título (como género, plataforma, duración estimada) y atributos derivados del comportamiento generalizado de los usuarios, con el fin de explorar su agrupamiento y potencial uso en sistemas de recomendación.

Para ello, aplicamos técnicas de preprocesamiento, binarización de atributos categóricos, análisis exploratorio, clustering no supervisado (mediante K-Means y DBSCAN), reducción de dimensionalidad con PCA, y validación interna de los agrupamientos mediante métricas como el coeficiente de Silhouette.

El objetivo general de este informe es entonces explorar el dataset desde una perspectiva de minería de datos, utilizando herramientas algorítmicas para comprender tanto las dinámicas sociales que emergen del uso compartido de videojuegos, como la estructura interna que permite agruparlos de forma significativa en función de la experiencia de juego.

## 2. Exploración de datos

Se presenta un análisis exploratorio del dataset de juegos de *HowLongToBeat*, centrado en entender las relaciones entre reseñas, tiempo de juego y variables complementarias. El dataset contiene 108 columnas con métricas de participación de usuarios, tiempos de completado, reseñas, plataformas y metadatos de juegos.

### Principales características del dataset

Se analizaron las principales características del dataset original, con los siguientes resultados:

| Métrica                                               | Valor     |
|-------------------------------------------------------|-----------|
| Total de juegos                                       | 147.474   |
| Juegos sin reviews                                    | 89.939    |
| Promedio de `game_review_score` (con reviews)         | 61.74     |
| Promedio de duración principal (`game_comp_main_avg`) | 2.45 horas |
| Promedio de duración entre modos de juego (`game_comp_all_avg`)          |5.23 horas|

### Estadísticas descriptivas relevantes

Se seleccionaron siete atributos relevantes basados en su potencial explicativo y cobertura de diferentes aspectos del dataset: calidad percibida, interacción de los usuarios, y duración del juego en horas.

| Variable               | Media  | Mediana | Desvío estándar | Mínimo | Máximo     |
| ---------------------- | ------ | ------- | --------------- | ------ | ---------- |
| `game_review_score`    | 24.26  | 0.0     | 32.38           | 0.0    | 100.00     |
| `review_review_count`  | 15.81  | 0.0     | 138.90          | 0.0    | 7439.00    |
| `game_count_replay`    | 1.86   | 0.0     | 22.34           | 0.0    | 2165.00    |
| `game_count_retired`   | 10.49  | 0.0     | 76.40           | 0.0    | 5227.00    |
| `game_count_comp`      | 49.52  | 1.0     | 476.26          | 0.0    | 36371.00   |
| `game_comp_all_avg`    | 5.23 h | 0.0 h   | 58.81 h         | 0.0 h  | 8105.08 h  |
| `game_invested_mp_avg` | 1.51 h | 0.0 h   | 51.77 h         | 0.0 h  | 10000.00 h |

De la tabla se puede ver la siguiente información:

- La mayoría de los juegos tienen un puntaje de reseña bajo o nulo (game_review_score y review_review_count con mediana = 0), lo que sugiere una alta cantidad de juegos poco evaluados por la comunidad.

- El promedio de game_count_comp es relativamente alto comparado con su mediana, lo que indica una distribución fuertemente sesgada: pocos juegos muy populares concentran la mayoría de las finalizaciones.

- Las métricas de tiempo (game_comp_all_avg y game_invested_mp_avg) presentan desviaciones estándar elevadas y valores máximos extremos, lo que indica gran variabilidad en duración de juegos, desde experiencias muy breves hasta compromisos prolongados.

- La baja mediana en casi todas las variables sugiere que la mayoría de los juegos tienen poca interacción o visibilidad, y solo un subconjunto reducido concentra el interés de los usuarios.

Este análisis justifica la necesidad de filtrar y escalar las métricas en etapas posteriores del procesamiento, especialmente para tareas de modelado o visualización.

### Análisis de atributos relacionados con "review"

Los atributos asociados a las reseñas de usuarios constituyen variables clave en este conjunto de datos. Estos campos, identificables por la raíz léxica "review", cuantifican tanto el volumen como la valoración de las evaluaciones recibidas por cada juego. El campo game_review_score representa el puntaje promedio de reseñas que tiene un juego, con valores que oscilan entre 0 y 100.

Adicionalmente, existen atributos con el formato review_X, donde X es un múltiplo de 5 entre 5 y 100 (por ejemplo, review_5, review_10, ..., review_100). Cada uno de estos atributos indica la cantidad de reseñas que otorgaron exactamente ese puntaje al juego. El campo review_review_count representa el número total de reseñas recibidas por el juego.

A continuación, se presenta una tabla que agrupa los juegos según intervalos de puntaje promedio (game_review_score) y muestra la cantidad de juegos que caen en cada intervalo. Además, se incluye la cantidad total de juegos que no han recibido ninguna reseña.

| #  | Intervalo de Puntaje | Cantidad de Juegos |
|----|----------------------|---------------------|
| 0  | 0-10                 | 90327               |
| 1  | 10-20                | 1117                |
| 2  | 20-30                | 1725                |
| 3  | 30-40                | 2385                |
| 4  | 40-50                | 4470                |
| 5  | 50-60                | 8526                |
| 6  | 60-70                | 14750               |
| 7  | 70-80                | 15085               |
| 8  | 80-90                | 6709                |
| 9  | 90-100               | 1547                |

Tambien se puede apreciar la siguiente distribución de los puntajes.

| Intervalo       | % Juegos | Interpretación               |
|-----------------|----------|------------------------------|
| 0 (Sin reseñas) | 61.17%    | Posible ruido para análisis  |
| 0-50            | 6.66%    | Recepción negativa           |
| 50-80           | 26.01%    | Recepción moderada           |
| 80-100          | 6.16%    | Excelente recepción          |

Se observa que existe una gran cantidad de juegos no han recibido ninguna review. Incluir estos juegos en el dataset puede resultar problemático si se desea realizar análisis que relacionen el puntaje promedio de reviews con otros atributos. Esto se debe a que, al no tener ninguna reseña, su puntaje promedio se representa como 0, lo cual no refleja una calificación real y puede introducir ruido o sesgos en el análisis. Por lo tanto, puede ser recomendable excluir estos registros en estudios donde las reviews son una variable clave. A continuación se puede apreciar la diferencia entre tener o no los juegos sin reviews.

<img src="img/juegos_vs_juegos_con_review.png" alt="juegos_vs_juegos_con_review" style="height:350px;"/>

Otro aspecto relevante del análisis es la relación entre la duración promedio de los juegos y su puntaje de reseñas. Para estudiar esta relación, se agruparon los juegos según su duración promedio (`game_comp_all_avg`), excluyendo aquellos que no cuentan con reseñas válidas. Esta decisión se justifica en que los juegos sin reseñas presentan un puntaje promedio de `0`, lo que no representa una calificación real y puede distorsionar los resultados.

La siguiente tabla muestra la distribución de juegos por intervalo de duración:


#### Cantidad de juegos por intervalo de duración (en horas)

| Intervalo de Duración | Cantidad de Juegos |
|------------------------|--------------------|
| 0-50 hrs               | 55,002             |
| 50-100 hrs             | 1,488              |
| 100-200 hrs            | 462                |
| 200-500 hrs            | 192                |
| 500-1000 hrs           | 67                 |
| >1000 hrs              | 54                 |


Como se observa, la gran mayoría de los juegos se concentran en el primer intervalo (menos de 50 horas), lo cual sugiere que el análisis visual con todos los datos puede estar fuertemente sesgado por outliers con duraciones extremadamente altas.

La siguiente visualización muestra dos gráficos de dispersión que relacionan la duración promedio de juego con su puntuación promedio de reseñas:

- El primero incluye todos los juegos válidos (sin reviews nulas),
- El segundo muestra el 95% inferior de los datos, eliminando outliers extremos.

<img src="img/puntuacion_vs_duracion_outlier.png" alt="puntuacion_vs_duracion_outlier" style="height:350px;"/>


Como puede apreciarse, al eliminar los valores atípicos (gráfico derecho), los puntos se agrupan de forma más compacta y se vuelve más evidente la estructura de los datos. Esta limpieza permite observar mejor la densidad y distribución de los juegos según su duración, revelando posibles patrones en su relación con la puntuación.

En particular, se nota que los juegos más extensos dentro del rango no atípico (hasta aproximadamente 100 horas) tienden a concentrarse en un rango de puntuaciones relativamente alto, aunque no necesariamente máximo. Esta observación sugiere una posible correlación entre mayor tiempo de juego y recepción más favorable por parte de los usuarios, aunque dicha correlación no es estrictamente lineal ni concluyente a simple vista.

Cabe destacar que en el gráfico sin outliers también se percibe una mayor variedad de puntuaciones en juegos de duración media (20–60 horas), lo que podría estar asociado a una mayor diversidad de géneros o experiencias ofrecidas en ese rango.

### Reducción de dimensiones

El conjunto de datos original contenía una gran cantidad de columnas, muchas de las cuales resultaban redundantes, irrelevantes o difíciles de interpretar. Con el objetivo de simplificar el análisis y mejorar la eficiencia de los modelos posteriores, se aplicó un proceso de reducción de dimensionalidad. Este proceso consistió en descartar o agrupar columnas según su utilidad y significado. Las decisiones principales fueron las siguientes:

- **Eliminación de columnas con significado desconocido:**
  Al haber sido recolectado mediante scraping, el dataset incluía 15 columnas cuyo propósito no pudo ser identificado. Estas columnas carecían de nombre descriptivo, contexto o documentación, lo que impedía su correcta interpretación y análisis. Por esta razón, se descartaron completamente.

- **Eliminación de columnas con información no procesable:**
  Se identificaron 6 columnas que contenían enlaces a imágenes, perfiles de usuario, páginas externas o texto libre sin una estructura estandarizada (como descripciones o títulos alternativos). Debido a que esta información no era útil en el contexto del análisis cuantitativo, también se eliminaron.

- **Agrupación de columnas de distribución de puntuaciones:**
  El dataset contenía 20 columnas del tipo `review_X`, donde `X` representa distintos umbrales de puntuación (5, 10, 15, ..., 100). Estas columnas representaban la cantidad de reseñas que asignaron una puntuación dentro de un cierto rango. Para reducir la dimensionalidad sin perder información significativa, se agruparon en cinco intervalos más amplios:

  - `review_20`: puntuaciones entre 1 y 20
  - `review_40`: puntuaciones entre 21 y 40
  - `review_60`: puntuaciones entre 41 y 60
  - `review_80`: puntuaciones entre 61 y 80
  - `review_100`: puntuaciones entre 81 y 100

  Esta transformación redujo las 20 columnas originales a solo 5, manteniendo la esencia de la distribución de puntuaciones y facilitando tanto la visualización como el procesamiento estadístico posterior.

En conjunto, estas acciones permitieron depurar significativamente el conjunto de datos, enfocando el análisis en variables relevantes y mejorando la interpretabilidad sin comprometer la calidad de la información.


## 3. Preguntas y problemas

A partir del set de datos y la exploración de este, se ha llegado a plantear las siguiente preguntas:

#### 1. ¿Qué características (género, tiempo de juego, rating) predicen mejor si un juego tendrá clasificación ESRB "M" (Maduro) frente a otras (ej: "T" para Teen)? ¿Puedo predecir cualquier clasificación de un juego basado en estos atributos? 

La clasificación de un juego determina el público objetivo de este. Entender qué características de un juego correlacionan con esto ayudará a la hora del desarrollo de este. Hipótesis: Juegos con etiquetas como Horror o Acción serán etiquetados como 'M'.  Así mismo, juegos con mayores tiempo de completitud serán de rating al menos M. Entonces, debería ser posible determinar el rating a partir de estas variables.

#### 2. ¿Podemos predecir el promedio de reviews de un juego en función de su tasa de abandono?

Los reviews de un juegos son un indicador de la recepción de este por parte de los jugadores, si un juego tiene altas tasas de abadono ¿repercute directamente en la review que reciba? Hipótesis: Los juegos con mayor tasa de abandono tendrán menores promedios de reviews que los con menos abandono.

#### 3. ¿Existen grupos de juegos con patrones similares que permitan agruparlos para recomendaciones?

Idintificar grupos entre los juegos para catalogarlos como 'similares' entre sí y poder ser recomendados juntos. Hipótesis: que se formen grupos entre juegos más rápidos de pasar y con menor rating, o juegos que tienen alto abandono y pertenecen a ciertos géneros.

#### 4. ¿Existen combinaciones de géneros o características (ej: "RPG + Open World") que correlacionen con tasas de abandono excepcionalmente altas/bajas?

Se busca encontrar características que influyan en el abandono de un juego. Hipótesis: Combinaciones como 'RPG + Open World' tendrán tasa de abandono significativamente mayor que 'RPG lineal', debido a la fatiga por contenido repetitivo. Otro como 'puzzle + indie' deben tener menos abandono quizás relacionado con su tiempo de completación.

**Para el desarrollo del hito 2 se tomó la decisión de no continuar con la primera pregunta ya que resulta redundante y no aporta suficiente información en comparación de las demás preguntas. La pregunta de qué características predicen una clasificación ‘M’ es más relevante para quienes están interesados en cumplir con los requisitos regulatorios o comerciales de un mercado específico, pero no necesariamente en el análisis profundo de las preferencias y tendencias de los jugadores.**

## 4. Propuesta Metodológica Experimental

#### Pregunta - promedio de reviews a partir de tasa de abandono y rejugabilidad

Para responder la pregunta *¿Es posible predecir el promedio de reviews a partir de la tasa de abandono y rejugabilidad?*, planteamos un enfoque supervisado
Preprocesamiento
- Se mantiene el preprocesamiento mencionado anteriormente que ayuda a la limpieza de datos y reducción de dimensionalidad.
- Dejaremos solo las columnas que nos sirvan, en este caso, las de cantidad de abandonos, las de rejugabilidad, las de completitud, las totales, las cantidades de reviews y el promedio de reviews.
- Buscaremos un threshold para eliminar las filas que no contengan reviews y las que no tengan muy poca informacion sobre sus datos de abandonos y rejugabilidad.

Modelo y técnica de minería de datos a utilizar

Se utilizará un enfoque supervisado, ya que tenemos una variable objetivo conocida y queremos predecirla utilizando la tasa de abandono/rejugabilidad para ello se investigará la efectividad de
- Regresión lineal como modelo base para tener una primera aproximación a la relación entre las variables.
- Random Forest para identificar el caso en que se tenga una relación no lineal o interacciones entre las variables predictoras.

Criterio de evaluación de los resultados
- R² como medida de calidad global
- MSE como medida de error cuadrático medio para evaluar la precisión del modelo.


#### Pregunta - Grupos de juegos con patrones similares para recomendaciones.

Para responder la pregunta *¿Existen grupos de juegos con patrones similares que permitan agruparlos para recomendaciones?*, aplicaremos un enfoque no supervisado basado en clustering

Preprocesamiento necesario adicional
- Se mantiene el preprocesamiento mencionado anteriormente que ayuda a la limpieza de datos y reducción de dimensionalidad.
- Se define la tasa de reseñas como el cociente entre el número de reseñas de un juego con el número total de jugadores.
- Se toma el percentil 50 de la tasa de reseñas para asegurar que los juegos a recomendar cumplan con un mínimo de calidad.
- Se eliminan aquellos juegos sin jugadores y sin reseñas.

Modelo y Técnica de minería de datos a utilizar
- Método del codo para obtener un k apropiado en k-means
- Se utilizará un enfoque no supervisado basado en clustering, esto porque no contamos con etiquetas predefinidas de "juegos recomendables juntos". Se evaluará con el algoritmo de k-means esperando una respuesta de grupos homogéneos, comparar con varios valores de k. Además se comparará con DBSCAN, que encontrará un número de clusters y permitirá contrastar en caso de que los grupos no sean homogéneos ni con una forma esférica.
- Se comparará el comportamiento del modelo con el dataset preprocesado + tasa de reseñas + generos con una versión del dataset que solo incluye los generos. 

Criterio de evaluación de los resultados
- Se utilizará Silhouette Score para ver que tan separados y cohesionado está un punto en su cluster y así se comparará con DBSCAN. Esto permitirá evaluar el comportamiento para ambas instancias del dataset y evaluar que los clusters generados correspondan efectivamente a grupos cohesionados y separados.


#### Pregunta - Combinaciones de géneros o características que correlacionen con tasas de abandono expecionalmente bajas/altas?

Para responder la pregunta *¿Existen combinaciones de géneros o características (ej: "RPG + Open World") que correlacionen con tasas de abandono excepcionalmente altas/bajas?*, aplicaremos un enfoque no supervisado basado en clustering

Preprocesamiento necesario
- Convertir game_profile_genre en variables binarias, permitiendo detectar combinaciones como "RPG + Open World"
- Definición de tasa de abandono: $\text{Tasa de abandono} = \frac{\text{game\_count\_retired}}{\text{game\_count\_total}}$
- Filtrado de juegos con baja cantidad de usuarios: Para evitar sesgos por juegos con pocos registros, se puede eligir un umbral mínimo de 10 usuarios.
- Agrupación de características adicionales:
Incluir variables binarias para: plataforma principal (game_profile_platform), modos de juego (game_comp_lvl_sp, co, mp), tipo de juego (game_game_type). Esto permitirá generar un análisis más granular y con mayor control en los datos.

Modelo y Técnica de minería de datos a utilizar
Se aplicará un enfoque no supervisado sobre las combinaciones de géneros y características jugables para encontrar patrones asociados a tasas de abandono. Es no supervisado dado que no conocemos combinaciones de antemano que aseguren una correlacion con tasas de abandono.
- Se utilizará K-Means para evaluar agrupaciones homogéneas en torno a tasas de abandono similares.
- Se probará también con DBSCAN como técnica alternativa, especialmente útil si los grupos son irregulares o no esféricos.

Criterio de evaluación de los resultados
- Se utilizará el Silhouette Score para evaluar la coherencia interna de los clusters formados.

## 5. Resultados

### Pregunta 3 - ¿Existen grupos de juegos con patrones similares que permitan agruparlos para recomendaciones?

#### Análisis Preliminar de los Datos

Dada la naturaleza de la pregunta, corresponde establecer un límite inferior tanto para el número de reviews mínimo, como para un score aceptable para generar recomendaciones. Para encontrar estos valores primero evaluaremos el mínimo de reviews que tiene que tener un juego para ser considerado en el análisis.

Para garantizar la validez de los grupos recomendados, se introdujo la métrica tasa de reseñas, definida como el cociente entre el número de reseñas y el número total de jugadores registrados para cada título. Esta medida evita sesgos provocados por juegos muy populares pero poco valorados, o viceversa.

Luego la tasa de reseñas inicial fue:
<p align="center"> <img src="img_p3/hist_reseñas.png" width="550"> </p>

Tras observar una gran cantidad de juegos con tasa nula o muy baja, se analizaron los percentiles. Se optó por conservar únicamente los juegos ubicados desde el percentil 50 hacia arriba, es decir, aquellos cuya tasa de reseñas es igual o superior a 0.215. Esto asegura una base más robusta para el análisis.

<p align="center"> <img src="img_p3/reseñas_filtrado.png" width="550"> </p>

Por otra parte, el atributo de generos originalmente venía en formato de strings, lo cual dificultaba su tratamiento. Se procedió a transformarlo en una representación binaria, obteniendo asi un dato que se podia trabajar de mejor manera.

Esta transformación permitió utilizar directamente la pertenencia de un juego a uno o varios géneros como variables de entrada para los algoritmos de agrupamiento.

#### Entrenamiento y evaluación del modelo

Realizamos el proceso de entrenamiento y evaluación de ambos modelos en base a dos grandes grupos, en el primero se tomo en consideracion el conjunto de todos los atributos, y en el segundo el agrupamiento por generos

1. **Agrupamiento con Todos los Atributos**

Se aplicaron los algoritmos KMeans y DBSCAN sobre un conjunto de datos completo (que incluye métricas como la tasa de reseñas y variables de interacción), previa reducción de dimensionalidad mediante PCA. Se identificó un punto de inflexión alrededor de 10 clusters con el método del codo.

<p align="center"> <img src="img_p3/clusters1.png" width="750"> </p>


Los resultados obtenidos fueron:

- Silhouette Score KMeans: 0.811

- Silhouette Score DBSCAN: -0.75

- DBSCAN - puntos de ruido: más de 31.000

Esto sugiere que KMeans captura mejor las estructuras subyacentes en los datos completos, generando agrupaciones cohesionadas.

2. **Agrupamiento Basado Solo en Géneros**

Posteriormente se repitió el procedimiento considerando únicamente las columnas de generos binarizados como atributos de entrada. Aunque el método del codo no mostró un punto de inflexión claro, se utilizó un valor de 𝑘=35 para KMeans.

<p align="center"> <img src="img_p3/clusters2.png" width="750"> </p>


Resultados obtenidos:

- Silhouette Score KMeans: 0.384

- Silhouette Score DBSCAN: 0.648

- DBSCAN - puntos de ruido: 3.805

En este enfoque, aunque los scores fueron más bajos en general, se identificaron grupos temáticamente coherentes, lo cual valida parcialmente su utilidad en contextos específicos.

#### Interpretación de Clusters
Para validar los agrupamientos obtenidos, se seleccionaron ejemplos específicos de clusters generados con ambos algoritmos. En el caso de KMeans sobre el conjunto completo, el cluster 12 agrupó juegos con una fuerte carga narrativa, de aventura o misterio, con una clara orientación a experiencias interactivas en tercera persona. DBSCAN, por su parte, identificó en su cluster 12 una cohorte de novelas visuales japonesas, y títulos centrados en narrativa lineal.

Esta coherencia temática sugiere que los clusters no solo tienen sentido desde una perspectiva estadística, sino también desde la experiencia empirica del jugador.

#### Conclusiones Preliminares

El análisis exploratorio realizado, sustentado en la binarización de géneros y filtros de reseña y puntajes, permitió identificar agrupamientos coherentes entre titulos con perfiles de genero similares. Esta estrategia demostró ser efectiva para estructurar el conjunto de datos de manera significativa, enfocandose en juegos que, por su nivel de recepción y evaluación, representan un subconjunto más confiable del universo total.

La aplicación del Método del Codo para ambos métodos implicó un análisis distinto, en primer lugar para el análisis completo se tiene una gráfica con una pendiente pronunciada que alcanza un punto de inflexión cercano a 10 clusters mientras que para el método que sólo considera los géneros se tiene una curva más suave que no presentó un punto de inflexión claramente definido, en este caso se considero que 35 clusters corresponde a un valor óptimo. Estas cantidades lograron equilibrar tanto la representación de la diversidad de combinaciones de genero, como la cohesión interna dentro de cada grupo. Clusters como el seleccionado nos dan cuenta de la autenticidad de los resultados obtenidos, ya que agrupan videojuegos con una gran carga narrativa, acción y tercera persona. Esto nos da cuenta de la utilidad del enfoque basado en géneros binarios como una herramienta potencial para sistemas de recomendación.

En cuanto a las técnicas de agrupamiento, pimero tenemos la ejecución dónde se consideró toda la data que evidencia una buena relación de clusters luego de entregando un silhouette score para kmeans de 0.811 y para dbscan de -0.75. En el análisis exclusivo de los géneros se obtiene un silhouette score de 0.38 para kmeans y 0.65 para dbscan. Esta información nos permite concluir que el mejor análisis para recomendar juegos se debe realizar utilizando kmeans con todos los datos, ya que si se compara con los clusters obtenidos en kmeans/dbscan en el caso de los generos podemos observar a través de los gráficos que no logra identificar correctamente los grupos.

#### Posibles Mejoras

Una de las mejoras más relevantes consiste en ampliar el espacio de características más allá de la información proporcionada, incorporar variables adicionales permitiría capturar aspectos que hoy quedan fuera del análisis. Por ejemplo, integrar métricas como el tiempo promedio de speedrun, su desviación estándar o incluso el tipo de final alcanzado, podría diferenciar entre juegos de ritmo vertiginoso y aquellos con un desarrollo más pausado, incluso dentro de un mismo género.

A la vez, añadir información contextual sobre las plataformas aportaría una capa adicional sobre la experiencia de juego. La inclusión de análisis de sentimiento o técnicas de extracción de temas a partir de reseñas y descripciones textuales podría reflejar la percepción real de la comunidad, identificando atributos más subjetivos que no son captados por los géneros formales. Esto ayudaría, por ejemplo, a distinguir entre un plataformero clásico con un juego del estilo metroidvania, que aunque comparten aspectos similares, sus experiencias de juego son en escencia muy diferentes.

En cuanto a los algoritmos, explorar con modelos jerárquicos podría revelar relaciones más finas entre géneros y subgéneros, ya que con los modelos actuales no queda tan claro el punto de inflexión en juegos que tengan una combinación de generos altamente cohesionados, lo que si es mucho mas claro al considerarlos jerarquicamente.

Por ultimo,  incluir encuestas a jugadores para verificar que los clusters resultantes no solo sean coherentes desde un punto de vista estadístico, sino también relevantes y útiles desde la experiencia del usuario.

### Pregunta 4 - ¿Existen combinaciones de géneros o características (ej: "RPG + Open World") que correlacionen con tasas de abandono excepcionalmente altas/bajas?

#### Análisis preliminar
Para abordar este problema, será necesario utilizar matrices binarias al trabajar con variables categóricas como género, plataforma y tipo, ya que las dos primeras están representadas como cadenas de texto (strings). Esto implica un procesamiento adicional para evitar redundancias o solapamientos entre categorías similares.

Inicialmente, se identificaron 247 géneros únicos en el conjunto de datos de videojuegos. Tras un proceso de normalización manual —agrupando géneros similares y eliminando aquellos con menos de 5 apariciones— este número se redujo a 88 géneros relevantes. Este filtro asegura que solo se consideren géneros significativos para el análisis, reduciendo el ruido del conjunto.

Luego se define un criterio de tasa de abandono (*retired rate*), esta tasa representa la cantidad de jugadores que han abandonado un juego y se calcula como:

$$
\text{retired\_rate} = \frac{\text{count\_retired}}{\text{count\_total}}
$$

**Donde**:
- `count_retired`: Número de jugadores que abandonaron el juego.
- `count_total`: Número total de jugadores analizados.

**Criterio de abandono alto**:
Un juego se clasifica como de *alto abandono* si su tasa supera el **20%** (`retired_rate > 0.2`). Esta condición se traduce a una variable binaria:
- `1`: Alto abandono (cumple la condición).
- `0`: Abandono bajo o normal.

Esta métrica es clave para evaluar patrones entre los *clusters* (género, plataforma, etc.) y la tendencia al abandono. El objetivo final es identificar relaciones que expliquen por qué ciertos juegos tienen tasas de abandono más elevadas.

#### Reducción de dimensionalidad

El dataframe actual presenta un problema de **alta dimensionalidad** debido a la naturaleza de los datos. Al transformar las variables categóricas (*géneros*, *plataformas* y *tipos de juego*) en matrices binarias (one-hot encoding), el conjunto de datos resultante contiene un total de **217 columnas/features**. Para abordar este desafío, se implementaron **tres técnicas de reducción de dimensionalidad**:

1. **PCA (Análisis de Componentes Principales)**

2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**

3. **UMAP (Uniform Manifold Approximation and Projection)**

**Metodología de evaluación**:
- Para cada técnica, se probaron múltiples configuraciones:
  - Variación en número de componentes
  - Diferentes hiperparámetros

En este informe se presentan solo los parámetros elegidos donde se consideró mejor rendimiento, esto debido a limitaciones computacionales que era hacer cross-validation y probar más configuraciones cada vez que se quería alterar el código.



#### Selección de atributos de k-means

Para lograr seleccionar correctamente los atributos de k-means para cada caso, se ejecutó el clustering para k en el intervalo [1, 11], para luego decidir el valor óptimo de k utilizando el método del codo basándose en los siguientes gráficos.

<img src="img_p4/codo_pca.png" alt="Imagen 1" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/codo_umap.png" alt="Imagen 2" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/codo_t-sne.png" alt="Imagen 3" width="300" style="display:inline-block;"/>

#### Selección de atributos de DBSCAN

Para determinar un valor adecuado de eps, se utilizó una gráfica de distancias al k-ésimo vecino más cercano (con k = min_samples).

<img src="img_p4/umbral_pca.png" alt="Imagen 4" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/umbral_umap.png" alt="Imagen 5" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/umbral_t-sne.png" alt="Imagen 6" width="300" style="display:inline-block;"/>

Donde el valor marcado como "umbral" es el que se tiene que definir como radio.

#### Evaluación de clusters

Hasta este momento se han definido un total de 6 clusters, los cuales serán evaluados con las siguientes métricas

- Silhouette score: cohesión interna del cluster.

- Davies-Bouldin index: separación entre clusters (menor es mejor).

- Calinski-Harabasz index: relación varianza inter-cluster/intra-cluster.

A continuación se ven los resultados:

<img src="img_p4/evaluation.png" alt="Imagen 4" width="500" style="display:inline-block; margin-right:10px;"/>

Donde se ve claramente que el cluster "K-means con PCA" es superior en todas las métricas definidas, por lo tanto, es el que se continuará utilizando. Además, notar que "DBSCAN con PCA" no se logra armar ningún cluster, por lo tanto, no es evaluable.

#### Conclusiones Preliminares

Para el Análisis final es necesario aplicar las etiquetas generadas por el cluster seleccionado a el set de datos, luego de esto se puede analizar la distribución de juegos con alto abandono entre clusters, lo que lleva a las siguientes gráficas, donde el valor `high_retired` en 1 significa que es un juego con alto abandono.

<img src="img_p4/abandono1.png" alt="Imagen 4" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/abandono2.png" alt="Imagen 5" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/abandono3.png" alt="Imagen 5" width="300" style="display:inline-block; margin-right:10px;"/>
<img src="img_p4/abandono4.png" alt="Imagen 6" width="300" style="display:inline-block;"/>

Las tablas que generan los gráficos se ven a continuación.

| K=3         |     |     | K=5         |     |     | K=7         |     |     | K=10        |     |     |
|-------------|-----|-----|-------------|-----|-----|-------------|-----|-----|-------------|-----|-----|
| cluster     |  0  |  1  | cluster     |  0  |  1  | cluster     |  0  |  1  | cluster     |  0  |  1  |
| 0           |19706|2783 | 0           |11428|1543 | 0           |8329 |1094 | 0           |5527 |876  |
| 1           |17626|3399 | 1           |9757 |1453 | 1           |7952 |1155 | 1           |6936 |691  |
| 2           |27338|2166 | 2           |18246|1240 | 2           |10010|1010 | 2           |5308 |968  |
|             |     |     | 3           |10875|2465 | 3           |7217 |1764 | 3           |5381 |1480 |
|             |     |     | 4           |14364|1647 | 4           |9964 |1209 | 4           |10314|478  |
|             |     |     |             |     |     | 5           |13250|758  | 5           |6096 |789  |
|             |     |     |             |     |     | 6           |7948 |1358 | 6           |6793 |755  |
|             |     |     |             |     |     |             |     |     | 7           |5353 |919  |
|             |     |     |             |     |     |             |     |     | 8           |7602 |727  |
|             |     |     |             |     |     |             |     |     | 9           |5360 |665  |




Si bien este análisis no reveló un patrón concluyente que asocie clusters específicos con tasas de abandono excepcionalmente altas, logramos identificar un cluster dominante que muestra consistentemente una menor proporción de juegos clasificados como high_retired = 1. Este hallazgo preliminar es prometedor, ya que podría indicar una mayor retención de usuarios en los géneros o características que definen a este grupo. No obstante, para validar y profundizar en esta potencial relación, será fundamental aplicar herramientas estadísticas adicionales en futuros trabajos, lo que permitirá un estudio más exhaustivo de las combinaciones de géneros que favorecen la retención.

## Anexo
- Link a codigo de pregunta 2: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-2/pregunta_2.ipynb
- Link a codigo de pregunta 3: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-2/pregunta_3.ipynb
- Link a codigo de pregunta 4: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-2/pregunta_4.ipynb
- Link a codigo para la exploracion de datos: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-2/Exploracion_Datos_Hito_1.ipynb 
- Link a codigo para los resultados preliminares: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-1/resultados_prelim.ipynb